In [265]:
from scipy.optimize import linprog
import pandas as pd
import numpy as np
import math

In [266]:
# IF YOU DON'T WANT TO EDIT ANYTHING, THEN JUST PRESS 'RUN' AT THE TOP BAR, SELECT 'RUN ALL CELLS', AND SCROLL TO THE BOTTOM FOR RESULTS!!

#Some constants used throughout the code
#Would highly suggest you to not edit these

BASEPRICES = [20,40,40,30,40,50,30,40,40,40,40,40,40,50,50,60,60,70,100,50,200,30,30,60,60,30,50,70,80,30,50,70,40,50,30,30,30,30,30,20,30,20,20,30]

GOODSNAMES = ["Soft wood","Hard wood","Iron","Coal","Tools","Steel","Fertilizer","Dye","Glass","Lead","Oil","Rubber","Silk","Explosives","Sulfur","Clippers","Engines","Steamers","Automobiles","Coffee","Fine art","Fruit","Liqour","Luxury clothes","Luxury furniture","Meat","Opium","Porcelain","Radios","Sugar","Tea","Telephones","Tobacco","Wine","Electricity","Services","Transportation","Paper","Groceries","Grain","Furniture","Fish","Fabric","Clothes"]


In [267]:
#Import the Excel sheet containing the buildings/PMs used to produce each good
#Remember to edit this name to match the name of the excel sheet you have on your PC. By default we look for 'BuildingSheet.xlsx'
data = pd.read_excel("BuildingSheet.xlsx")

#Create the data frame based only on the buildigns we want included in the spreadsheet
df = data[data['Included'].astype(str).str.contains("1")]

#print(df)



In [268]:
#Construct the matrices needed for creating our linear minimizing problem (maths stuff)
#Optimization for labor is done per 100 pops

#Calculating the scalar factor for outputs and inputs when optimizing for construction
#Also taking into account potential construction bonuses (aka companies)
scalar_per_construction=df['Construction'].values*(1-df['ConBonus'].values)

a = np.lcm.reduce(scalar_per_construction.astype(np.int64))

scalar_factors_construction=a*np.reciprocal(scalar_per_construction)

#Change this if you want to change per how many pops you optimize for
per_labor = 100

#Calculating the scalar factor for outputs and inputs when optimizing for labor
scalar_per_labor=df['Labor'].values/per_labor

a = np.lcm.reduce(scalar_per_labor.astype(np.int64))

scalar_factors_labor=a*np.reciprocal(scalar_per_labor)

#Separating the inputs and outputs of all buildings included
df_inp=df.filter(like='Inp', axis=1)
df_out=df.filter(like='Out', axis=1)
#Taking into account potential throughput bonuses
output_bonuses=1+df['TBonus'].values

In [269]:
#Code for the function which preps all the math so that we can perform optimization!

def optimization_function(scalar_factors, inp, out, tbonus):
    #Scale the inputs and outputs based on the correct scalar dependent on the maximum consturction in data set
    #Fancy way of saying that because we need to add and subtract different goods' outputs and inputs from one-another
    #And we need to have all factors be whole numbers, we have to multiply all outputs and inputs such that this is possible!
    #Otherwise we couldn't make constraits like logging camps eq.eff. is equal to eq.eff. of steel mills (they have different construction)
    inp=inp.mul(scalar_factors, axis=0).astype(int)
    out=out.mul(scalar_factors, axis=0).astype(int)
    out=out.mul(tbonus, axis=0).astype(int)
    
    #print(df_out)
    #print(df_inp)
    
    #Combine inputs and outputs into one equivalent net dataframe
    df_eq_net=out.sub(inp.values)
    
    #print(df_eq_net)

    #Creating our function which we will minimize! 
    #Note: since we actually want to maximize but liner optimization (SciPy) is what it is, we just reverse the function and minimize instead!
    
    df_obj_func=df_eq_net.cumsum()*-1
    
    #print(df_obj_func)
    
    c=df_obj_func.iloc[len(tbonus)-1].values
    
    #print(c)

    #Creating the lhs adn rhs matrices used in our linear optimization problem (also the whole reason this program has been written)
    #And since we want the eq.eff. to be equal for all buildings, all lhs will naturally equal 0
    
    A=[]
    
    rhs=[]
    
    for i in range(len(scalar_factors)):
        for j in range(i+1,len(scalar_factors)):
            A.append(df_eq_net.iloc[i].values-df_eq_net.iloc[j].values)
            rhs.append(0)

    return c, A, rhs

In [270]:
#Calling the function for both construction and labor! Feel free to comment the other out if you're not insterested in the results

c_con, A_con, rhs_con = optimization_function(scalar_factors_construction, df_inp, df_out, output_bonuses)

c_lab, A_lab, rhs_lab = optimization_function(scalar_factors_labor, df_inp, df_out, output_bonuses)

In [271]:
#Creating the bounds for each good
#Aka set it so that we don't get steel prices at like 1000+, but rather keep the prices in the +-75% range
#These are the same regardless of whether we optimize for labor or for construction

boundaries=[]

for bp in BASEPRICES:
    boundaries.append((bp*0.25, bp*1.75))

#print(boundaries)

In [272]:
#Optimization!

res_con=linprog(c_con, A_eq=A_con, b_eq=rhs_con, bounds=boundaries)
res_con.fun

res_lab=linprog(c_lab, A_eq=A_lab, b_eq=rhs_lab, bounds=boundaries)
res_lab.fun

-7738980.150021586

In [273]:
#Making the optimal prices per construction more readable

procentages=[]

for i in range(len(res_con.x)):
    procentages.append(str(round(((res_con.x[i]/BASEPRICES[i])-1)*100, 1))+"%")

d = {'Good':GOODSNAMES, 'Base price':BASEPRICES, 'Optimal price per construction':res_con.x, 'Procentage':procentages}
readable_df=pd.DataFrame(data=d)
readable_df


,Good,Base price,Optimal price per construction,Procentage
0,Soft wood,20,18.736460,-6.3%
1,Hard wood,40,10.000000,-75.0%
2,Iron,40,45.809801,14.5%
3,Coal,30,36.647841,22.2%
4,Tools,40,54.735604,36.8%
5,Steel,50,87.500000,75.0%
6,Fertilizer,30,7.500000,-75.0%
7,Dye,40,10.000000,-75.0%
8,Glass,40,10.000000,-75.0%
9,Lead,40,10.000000,-75.0%


In [274]:
#Making the optimal prices per labor more readable

procentages=[]

for i in range(len(res_lab.x)):
    procentages.append(str(round(((res_lab.x[i]/BASEPRICES[i])-1)*100, 1))+"%")

d = {'Good':GOODSNAMES, 'Base price':BASEPRICES, 'Optimal price per labor':res_lab.x, 'Procentage':procentages}
readable_df=pd.DataFrame(data=d)
readable_df


,Good,Base price,Optimal price per labor,Procentage
0,Soft wood,20,34.387232,71.9%
1,Hard wood,40,10.000000,-75.0%
2,Iron,40,57.621305,44.1%
3,Coal,30,46.097044,53.7%
4,Tools,40,59.766918,49.4%
5,Steel,50,87.500000,75.0%
6,Fertilizer,30,7.500000,-75.0%
7,Dye,40,10.000000,-75.0%
8,Glass,40,10.000000,-75.0%
9,Lead,40,10.000000,-75.0%
